# Get Team Data
This notebook pulls team level data to compare and predict winners.

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import time
import datetime

from importlib import reload
import projections
reload(projections)

pd.options.display.max_colwidth = 200
pd.options.display.max_rows, pd.options.display.min_rows = 35, 35

In [2]:
reload(projections)
data = projections.GetData()

# Store soups for later if re-initialization of GetData is necessary
year_soup = data.year_soup
off_soup = data.off_soup
def_soup = data.def_soup

## Get Data

In [19]:
reload(projections)
data = projections.GetData(year_soup=year_soup, off_soup=off_soup, def_soup=def_soup)

# Get current standings, team offensive & defensive stats
standings = data.team_standings()
team_offense = data.team_offense_stats()
team_defense = data.team_defense_stats()

## Create Models

In [21]:
standings

,id,division,wins,losses,win_loss_perc,points,points_opp,points_diff,mov,sos_total,srs_total,srs_offense,srs_defense
Miami Dolphins,mia,AFC East,8,3,.727,339,251,88,8.0,-3.1,4.9,9.1,-4.2
Buffalo Bills,buf,AFC East,6,6,.500,328,227,101,8.4,-3.4,5.0,4.6,0.3
New York Jets,nyj,AFC East,4,7,.364,163,238,-75,-6.8,0.7,-6.1,-7.2,1.1
New England Patriots,nwe,AFC East,2,9,.182,148,248,-100,-9.1,-0.4,-9.5,-10.1,0.6
Baltimore Ravens,rav,AFC North,9,3,.750,324,187,137,11.4,0.6,12.0,5.4,6.6
Pittsburgh Steelers,pit,AFC North,7,4,.636,182,205,-23,-2.1,3.1,1.0,-2.7,3.7
Cleveland Browns,cle,AFC North,7,4,.636,239,209,30,2.7,2.0,4.7,1.3,3.4
Cincinnati Bengals,cin,AFC North,5,6,.455,212,242,-30,-2.7,3.3,0.6,-0.3,0.9
Jacksonville Jaguars,jax,AFC South,8,3,.727,254,225,29,2.6,2.1,4.7,2.2,2.5
Indianapolis Colts,clt,AFC South,6,5,.545,269,268,1,0.1,0.5,0.6,4.5,-3.8
